<a href="https://colab.research.google.com/github/Mario-RJunior/calculadora-imoveis/blob/master/machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning

Agora é a etapa em que iremos gerar um modelo usando técnicas de Machine Learning para gerar um modelo que irá nos ajudar a prever o valor de aluguéis de imóveis da cidade de São Paulo. Para isso usaremos como base os arquivos csv gerados durante a [análise exploratória](https://github.com/Mario-RJunior/calculadora-imoveis/blob/master/analise_exploratoria.ipynb).

## 1) Importar os dados

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.metrics import r2_score

In [2]:
treino = pd.read_csv('https://raw.githubusercontent.com/Mario-RJunior/calculadora-imoveis/master/treino_preprocessado.csv')
teste = pd.read_csv('https://raw.githubusercontent.com/Mario-RJunior/calculadora-imoveis/master/teste_preprocessado.csv')

In [3]:
# Divisão para variáveis X e y
X_train = treino.drop(labels='preco', axis=1)
y_train = treino['preco']
X_test = teste.drop('preco', axis=1)
y_test = teste['preco']

## 2) Modelo Baseline

Inicialmente criaremos um modelo denominado de ***Baseline*** que servirá como parâmetro para comparar seu resultado com os dos outros modelos que iremos gerar. É importante ressaltar que para que um modelo seja considerado bom ele deve ser superior ao baseline.

Como modelo baseline usaremos um algorítmo de regressão linear.

In [4]:
# Criando o modelo de regressão linear
rl = LinearRegression()
rl.fit(X_train, y_train)
rl.score(X_test, y_test)

0.5617659641427768

Note que este modelo tem um score de aproximadamente 0.45, que é bem ruim. Agora, precisamos testar outros modelos para encontrar o melhor e mais adequado possível.

## 3) Testando outros modelos

### 3.1) K-Nearest Neighbors (KNN)

In [5]:
neigh = KNeighborsRegressor()
neigh.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [6]:
neigh.score(X_test, y_test)

0.6399890193124183

### 3.2) Random Forest

In [7]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [8]:
rf.score(X_test, y_test)

0.6419024223837924

### 3.3) Adaboost

In [9]:
regr = AdaBoostRegressor()
regr.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
                  n_estimators=50, random_state=None)

In [10]:
regr.score(X_test, y_test)

0.6011958998622478

Podemos agora, afim de otimizar nosso tempo criar um modelo para diversos algorítmos simultaneamente.

In [11]:
# Importando os estimadores
from sklearn.linear_model import RidgeCV, Lasso, ElasticNet, LassoLars, HuberRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [12]:
# Criando uma lista com todos os estimadores
reg_list = [RidgeCV(),
            LGBMRegressor(), 
            XGBRegressor(objective='reg:squarederror'),
            SVR(),
            GradientBoostingRegressor(),
            MLPRegressor()
            ]

In [13]:
# Criando o modelo
from sklearn.model_selection import cross_val_score
import numpy as np

for reg in reg_list:
    print(f'Treinando Modelo {reg.__class__.__name__}')
    reg.fit(X_train, y_train)
    
    train_score = reg.score(X_train, y_train)
    cv_scores = cross_val_score(reg, X_train, y_train)
    test_score = reg.score(X_test, y_test)
    
    print(f"R2 Score Train: {train_score}")
    print(f"R2 Score Valid: {np.mean(cv_scores):.2f} +- {np.std(cv_scores):.2f}")
    print(f"R2 Score Test: {test_score}")
    print('='*80)

Treinando Modelo RidgeCV
R2 Score Train: 0.694995494792588
R2 Score Valid: 0.69 +- 0.04
R2 Score Test: 0.5628158188285721
Treinando Modelo LGBMRegressor
R2 Score Train: 0.8335649897226411
R2 Score Valid: 0.73 +- 0.01
R2 Score Test: 0.6524018217807195
Treinando Modelo XGBRegressor
R2 Score Train: 0.8173085197618972
R2 Score Valid: 0.74 +- 0.01
R2 Score Test: 0.6748886425763778
Treinando Modelo SVR
R2 Score Train: 0.7331335431720392
R2 Score Valid: 0.72 +- 0.03
R2 Score Test: 0.6424482920301618
Treinando Modelo GradientBoostingRegressor
R2 Score Train: 0.8279163984754841
R2 Score Valid: 0.74 +- 0.01
R2 Score Test: 0.663573061815669
Treinando Modelo MLPRegressor


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


R2 Score Train: 0.7150139858715128
R2 Score Valid: 0.70 +- 0.05
R2 Score Test: 0.5895900503274301
